In [4]:
import pandas as pd

# 读取Excel文件
file_path = 'Q2指标.xlsx'
data = pd.read_excel(file_path)


In [5]:
data

,比赛ID,已进行时间,选手1的波动,选手2的波动,选手1的连胜盘分数,选手2的连胜盘分数,选手1的连胜局分数,选手2的连胜局分数,选手1的分差盘得分,选手2的分差盘得分,选手1的分差局得分,选手2的分差局得分,球员1本局是否胜利,球员2本局是否胜利
0,2023-wimbledon-1301,00:05:32,6,4,0,0,0,0,0,0,0,0,0,1
1,2023-wimbledon-1301,00:10:45,3,5,0,0,0,0,0,0,1,-1,1,0
2,2023-wimbledon-1301,00:13:26,4,1,0,0,0,0,0,0,0,0,0,1
3,2023-wimbledon-1301,00:17:08,0,4,0,0,0,0,0,0,1,-1,1,0
4,2023-wimbledon-1301,00:22:19,5,3,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7279,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0
7280,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0
7281,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,1
7282,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0


In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np



# 提取相关的评价指标列
criteria = data.iloc[:, 2:12]

# 权重设置
weights = np.array([0.34, 0.10, 0.23, 0.10, 0.23])  # 对应波动、连胜盘分数、连胜局分数、分差盘得分、分差局得分

# 分别处理选手1和选手2的数据
criteria_p1 = criteria.iloc[:, [0, 2, 4, 6, 8]]  # 选手1的指标
criteria_p2 = criteria.iloc[:, [1, 3, 5, 7, 9]]  # 选手2的指标

# 标准化处理
scaler = MinMaxScaler()
criteria_p1_scaled = scaler.fit_transform(criteria_p1)
criteria_p2_scaled = scaler.fit_transform(criteria_p2)

# 构建加权标准化矩阵
weighted_criteria_p1 = criteria_p1_scaled * weights
weighted_criteria_p2 = criteria_p2_scaled * weights

# 计算正理想解和负理想解
positive_ideal_p1 = weighted_criteria_p1.max(axis=0)
negative_ideal_p1 = weighted_criteria_p1.min(axis=0)
positive_ideal_p2 = weighted_criteria_p2.max(axis=0)
negative_ideal_p2 = weighted_criteria_p2.min(axis=0)

# 计算与正理想解和负理想解的距离
distance_to_positive_p1 = np.sqrt(((weighted_criteria_p1 - positive_ideal_p1) ** 2).sum(axis=1))
distance_to_negative_p1 = np.sqrt(((weighted_criteria_p1 - negative_ideal_p1) ** 2).sum(axis=1))
distance_to_positive_p2 = np.sqrt(((weighted_criteria_p2 - positive_ideal_p2) ** 2).sum(axis=1))
distance_to_negative_p2 = np.sqrt(((weighted_criteria_p2 - negative_ideal_p2) ** 2).sum(axis=1))

# 计算相对接近度
relative_closeness_p1 = distance_to_negative_p1 / (distance_to_positive_p1 + distance_to_negative_p1)
relative_closeness_p2 = distance_to_negative_p2 / (distance_to_positive_p2 + distance_to_negative_p2)

# 结果整合
scores_p1_p2 = pd.DataFrame({
    "选手1评分": relative_closeness_p1,
    "选手2评分": relative_closeness_p2
})

# 将计算出的评分加到原数据的后两列
data_with_scores = data.copy()
data_with_scores['选手1势头评分'] = scores_p1_p2['选手1评分']
data_with_scores['选手2势头评分'] = scores_p1_p2['选手2评分']


output_file_path = 'Q2指标_带势头评分.xlsx'
data_with_scores.to_excel(output_file_path, index=False)
